In [172]:
import os
print(os.getcwd())

/Users/liuqiaochu/Desktop/spring II/empirical finance/project


# Data clean and merge

- Dataset used in this notebook
- us_1991 to us_2022
- match.xlsx ( birdge file to match tri with gvkey)
- financial.csv ( scaling data)

In [236]:
# Directory containing the CSV files
directory = "/Users/liuqiaochu/Desktop/spring II/empirical finance/project/data1"

# List to hold all CSV file paths
csv_files = []

# Iterate over years from 1990 to 2022
for year in range(1991, 2023):
    # Construct the filename for each year
    filename = f"{year}_us.csv"
    # Append the full path to the CSV file to the list
    csv_files.append(os.path.join(directory, filename))

In [174]:
import pandas as pd

# List to hold all DataFrames
dfs = []

# Read each CSV file into a DataFrame, keeping only the specified columns, and append to the list
for file in csv_files:
    df = pd.read_csv(file, usecols=['1. YEAR', '2. TRIFD', '15. PARENT CO NAME', '104. TOTAL RELEASES'])
    dfs.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dfs, ignore_index=True)

# If you want to see the first few rows of the merged DataFrame
print(merged_df.head())
print(merged_df.tail())



   1. YEAR         2. TRIFD  15. PARENT CO NAME  104. TOTAL RELEASES
0     1991  73401TTLPTHIGHW  VALERO ENERGY CORP              14806.0
1     1991  73098KRRMC906SO      CVR ENERGY INC              34505.0
2     1991  73098KRRMC906SO      CVR ENERGY INC               3205.0
3     1991  73401TTLPTHIGHW  VALERO ENERGY CORP                120.0
4     1991  73401TTLPTHIGHW  VALERO ENERGY CORP               3030.0
         1. YEAR         2. TRIFD        15. PARENT CO NAME  \
2712514     2022  45242HBFLL4440M              HB FULLER CO   
2712515     2022  29406SDFNS5862N  US DEPARTMENT OF DEFENSE   
2712516     2022  64504VRCPR2250L                       NaN   
2712517     2022  2759WCRLNS1524L     THE SUNROCK GROUP INC   
2712518     2022  7704WCHMNL1414W                       NaN   

         104. TOTAL RELEASES  
2712514               0.0000  
2712515               0.0004  
2712516            1478.1400  
2712517              24.7237  
2712518               0.0000  


In [175]:
merged_df = merged_df.rename(columns={'1. YEAR': 'YEAR', '2. TRIFD': 'TRIFD','15. PARENT CO NAME':'PARENT CO NAME', '104. TOTAL RELEASES':'TOTAL RELEASES'})

In [176]:
sum_merged_df = merged_df.groupby(['YEAR', 'PARENT CO NAME'])['TOTAL RELEASES'].sum().reset_index()

In [177]:
print(sum_merged_df.head())
print(sum_merged_df.tail())

   YEAR         PARENT CO NAME  TOTAL RELEASES
0  1991  3A COMPOSITES USA INC          4250.0
1  1991                  3M CO      40739263.0
2  1991              3M CO INC       3424874.0
3  1991  3M TECHNICAL CERAMICS          7140.0
4  1991        3P HOLDINGS INC         77585.0
        YEAR           PARENT CO NAME  TOTAL RELEASES
192102  2022               ZOETIS INC     13875.44045
192103  2022              ZOLTEK CORP      2320.00000
192104  2022      ZSCHIMMER & SCHWARZ      1002.80000
192105  2022  ZSCHIMMER & SCHWARZ INC        23.00000
192106  2022       ZURN HOLDINGS INC.      3532.03000


In [178]:
import pandas as pd

# Read left and right files
merged_file = sum_merged_df
link = pd.read_excel('match.xlsx')

link = link.drop_duplicates(subset=['parent_name'])

# Merge the dataframes based on 'year' and 'parent_name'
match_merged_df = merged_file.merge(link, how='left', left_on=['PARENT CO NAME'], right_on=['parent_name'])

# Drop the 'parent_name' column from the merged DataFrame
#match_merged_df.drop(['year','parent_name'], axis=1, inplace=True)

# Print the merged DataFrame
print(match_merged_df)

#merged_df.to_csv('merged_df.csv')

        YEAR           PARENT CO NAME  TOTAL RELEASES    year parent_name  \
0       1991    3A COMPOSITES USA INC    4.250000e+03     NaN         NaN   
1       1991                    3M CO    4.073926e+07  1987.0       3M CO   
2       1991                3M CO INC    3.424874e+06  1987.0   3M CO INC   
3       1991    3M TECHNICAL CERAMICS    7.140000e+03     NaN         NaN   
4       1991          3P HOLDINGS INC    7.758500e+04     NaN         NaN   
...      ...                      ...             ...     ...         ...   
192102  2022               ZOETIS INC    1.387544e+04  2007.0  ZOETIS INC   
192103  2022              ZOLTEK CORP    2.320000e+03     NaN         NaN   
192104  2022      ZSCHIMMER & SCHWARZ    1.002800e+03     NaN         NaN   
192105  2022  ZSCHIMMER & SCHWARZ INC    2.300000e+01     NaN         NaN   
192106  2022       ZURN HOLDINGS INC.    3.532030e+03     NaN         NaN   

          cmpy    newid   Ucmpy  score    gvkey  
0          NaN      NaN  

In [179]:
sum_df = match_merged_df.groupby(['YEAR', 'gvkey'])['TOTAL RELEASES'].sum().reset_index()
sum_df.columns = ['YEAR', 'gvkey', 'SUM_TOTAL_RELEASES']
print(sum_df.tail())

       YEAR     gvkey  SUM_TOTAL_RELEASES
24794  2022  187740.0         7847.800000
24795  2022  189491.0         3969.764600
24796  2022  196159.0       302694.140000
24797  2022  274165.0       974525.610800
24798  2022  287462.0       220891.939229


In [76]:
sum_df.to_csv('sum_total_releases.csv', index=False)

In [180]:
financial = pd.read_csv('financial.csv')
financial_df = financial[financial['exchg'].isin([11, 14, 10])]
financial_df = financial_df.dropna(subset=['sic'])

financial_df.tail()

,gvkey,fyear,at,eq,ppent,sale,exchg,sic
30071,315318,2019.0,833.4,2220.9,325.9,1835.9,11,2890
30072,315318,2020.0,989.9,2319.8,296.4,1853.7,11,2890
30073,315318,2021.0,1214.5,2480.7,338.3,2399.8,11,2890
30074,315318,2022.0,1188.7,2332.5,362.3,2549.4,11,2890
30075,315318,2023.0,1197.5,2328.5,387.2,2333.2,11,2890


In [201]:
# Merge TRI & Compustat data
sum_df2 =financial_df.merge(sum_df, how='inner', left_on=['fyear', 'gvkey'], right_on=['YEAR', 'gvkey'])
sum_df2.drop('YEAR', axis=1, inplace=True)
sum_df2.dropna()

,gvkey,fyear,at,eq,ppent,sale,exchg,sic,SUM_TOTAL_RELEASES
0,1004,1991.0,289.537,196.737,60.422,422.657,11,5080,399249.00
1,1004,1992.0,265.036,189.216,56.052,382.780,11,5080,215104.00
2,1004,1993.0,307.723,189.488,54.783,407.754,11,5080,276913.00
3,1004,1994.0,321.632,197.119,56.596,451.395,11,5080,311536.00
4,1004,1995.0,338.012,204.635,54.831,504.990,11,5080,226010.00
...,...,...,...,...,...,...,...,...,...
15547,315318,2013.0,383.239,1032.595,139.670,745.951,11,2890,14078.12
15548,315318,2014.0,1578.400,2459.600,175.000,843.200,11,2890,8388.87
15549,315318,2015.0,2270.500,2103.900,491.600,2542.300,11,2890,7823.81
15550,315318,2016.0,2071.200,2736.100,460.500,3585.900,11,2890,11705.47


In [79]:
sum_df2.to_csv('sum_total.csv', index=False)

# Scaling

In [202]:
sum_df2['fyear'] = sum_df2['fyear'].astype(int)

In [238]:
# def ind_scaling(sum_df2, variable_name):
#     industry_avg_variable = sum_df2.groupby(['sic','fyear'])[variable_name].mean().reset_index()
#     industry_avg_variable.columns = ['sic', 'fyear', 'In_' + variable_name]

#     # Merge industry average back to the original DataFrame
#     sum_df2 = pd.merge(sum_df2, industry_avg_variable, on=['sic', 'fyear'], how='left')

#     # Calculate the scaling factor
#     sum_df2['p_' + variable_name] = sum_df2['SUM_TOTAL_RELEASES'] / sum_df2['In_' + variable_name]

#     new_df_variable_name = 'df_' + variable_name
#     globals()[new_df_variable_name] = sum_df2[['fyear', 'gvkey', variable_name, 'p_' + variable_name, 'In_' + variable_name]]

#     return globals()[new_df_variable_name]

# at_df = ind_scaling(sum_df2, 'at')
# eq_df = ind_scaling(sum_df2, 'eq')
# ppent_df = ind_scaling(sum_df2, 'ppent')
# sale_df = ind_scaling(sum_df2, 'sale')


,fyear,gvkey,at,p_at,In_at
0,1991,1004,289.537,1378.922210,289.537000
1,1992,1004,265.036,811.602952,265.036000
2,1993,1004,307.723,899.877487,307.723000
3,1994,1004,321.632,968.610089,321.632000
4,1995,1004,338.012,668.644900,338.012000
...,...,...,...,...,...
15547,2013,315318,383.239,15.841614,888.679667
15548,2014,315318,1578.400,6.662769,1259.066667
15549,2015,315318,2270.500,4.715269,1659.250000
15550,2016,315318,2071.200,7.408056,1580.100000


In [205]:
## Scaling
sum_df2['p_at'] = sum_df2['SUM_TOTAL_RELEASES']/sum_df2['at']
sum_df2['p_eq'] = sum_df2['SUM_TOTAL_RELEASES']/sum_df2['eq']
sum_df2['p_ppent'] = sum_df2['SUM_TOTAL_RELEASES']/sum_df2['ppent']
sum_df2['p_sale'] = sum_df2['SUM_TOTAL_RELEASES']/sum_df2['sale']

In [206]:
##generate new dataframe

at_df = sum_df2[['fyear', 'gvkey', 'at', 'p_at']]
eq_df = sum_df2[['fyear', 'gvkey', 'eq', 'p_eq']]
ppent_df = sum_df2[['fyear', 'gvkey', 'ppent', 'p_ppent']]
sale_df = sum_df2[['fyear', 'gvkey', 'sale', 'p_sale']]

In [207]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scaling - Asset

In [208]:
percentiles_df = at_df.groupby('fyear')['p_at'].quantile([0.2, 0.4, 0.6, 0.8]).unstack().reset_index()
percentiles_df.columns = ['fyear', 'p20', 'p40', 'p60', 'p80']

# Merge percentiles back into the original dataset
at_df = at_df.merge(percentiles_df, on='fyear', how='left')

In [209]:
print(at_df.tail())

       fyear   gvkey        at       p_at       p20        p40        p60  \
15547   2013  315318   383.239  15.841614  0.433306   8.503616  62.630610   
15548   2014  315318  1578.400   6.662769  0.534995   8.457236  74.689777   
15549   2015  315318  2270.500   4.715269  0.612331  10.468416  84.553539   
15550   2016  315318  2071.200   7.408056  0.364546  10.780810  88.182545   
15551   2017  315318  2340.600   4.616383  0.634837  11.070316  80.033291   

              p80  
15547  493.501640  
15548  508.895786  
15549  539.341568  
15550  515.532442  
15551  504.880403  


In [210]:
##################################
# Function to map e_at value to group tag
def map_to_group_tag(scale_value, percentiles):
    if scale_value <= percentiles['p20']:
        return 1
    elif scale_value <= percentiles['p40']:
        return 2
    elif scale_value <= percentiles['p60']:
        return 3
    elif scale_value <= percentiles['p80']:
        return 4
    else:
        return 5  # Adjust this based on your requirements

# Add a new column 'group_tag' to the DataFrame based on 'e_at' percentiles for each year
group_tags = []

for year in at_df['fyear'].unique():
    year_data = at_df[at_df['fyear'] == year]
    percentiles = year_data[['p20', 'p40', 'p60', 'p80']].iloc[0].to_dict()
    group_tags.extend(year_data['p_at'].apply(map_to_group_tag, args=(percentiles,)))

at_df['group_tag'] = group_tags


In [211]:
at_df.head()

,fyear,gvkey,at,p_at,p20,p40,p60,p80,group_tag
0,1991,1004,289.537,1378.922210,16.683231,125.940789,414.392345,1894.698128,4
1,1992,1004,265.036,811.602952,10.822616,95.586792,383.156290,1814.004272,2
2,1993,1004,307.723,899.877487,10.334283,92.647501,289.686822,1477.709719,1
3,1994,1004,321.632,968.610089,4.387066,62.109581,252.017840,1094.266391,2
4,1995,1004,338.012,668.644900,4.167967,48.314072,239.411707,1126.941572,4


In [212]:
###################################################################
# Count occurrences of each group for each year
group_counts = at_df.groupby(['fyear', 'group_tag']).size().unstack(fill_value=0)
print(group_counts)

group_tag    1    2    3    4    5
fyear                             
1991       111  109  128  106  116
1992       112  127  106  119  121
1993       124  124  111  102  120
1994       133  114   94  105  128
1995       103  111  118  116  119
1996       103  113  107  120  127
1997       115  106  116  105  112
1998       105  122  132   95  137
1999       123  108  106  114  116
2000       121   98  116  115  106
2001       103  114  108  113  119
2002       113  101  105  106  104
2003        94  117  107   92  120
2004       106  104  102  111  106
2005        91  109  112  107  103
2006       102   85   94  104  118
2007        99   95   94   98  107
2008        85   89  113   90  103
2009       108   91   82   96   94
2010        96   90   92   86  106
2011       112   90   78   87   98
2012        88   99   80  110   80
2013        86   92   95   94   83
2014        99   78   84   79   97
2015        78   74   81   96   88
2016        80   66   83   88   81
2017        77   66 

In [213]:
#########################################
# portfolio formation
# Add a new column 'trade' assigning 1 to Group 5 and -1 to Group 1 for each year
at_df['trade'] = at_df['group_tag'].apply(lambda x: 1 if x == 5 else (-1 if x == 1 else 0))
sum_at_l = at_df[at_df['trade'] == 1].groupby('fyear')['at'].sum()
sum_at_s = at_df[at_df['trade'] == -1].groupby('fyear')['at'].sum()
# calculate wights each year
for year in at_df['fyear'].unique():
    year_data = at_df[at_df['fyear'] == year]
    at_df.loc[at_df['fyear'] == year, 'weight_l'] = year_data['at']* (year_data['trade'] == 1) /sum_at_l.iloc[year-1991]
    at_df.loc[at_df['fyear'] == year, 'weight_s'] = -year_data['at']* (year_data['trade'] == -1) /sum_at_s.iloc[year-1991]
portfolio_at = at_df
portfolio_at.dropna(inplace=True)
portfolio_at.head(20)

,fyear,gvkey,at,p_at,p20,p40,p60,p80,group_tag,trade,weight_l,weight_s
0,1991,1004,289.537,1378.922210,16.683231,125.940789,414.392345,1894.698128,4,0,0.000000,-0.000000
1,1992,1004,265.036,811.602952,10.822616,95.586792,383.156290,1814.004272,2,0,0.000000,-0.000000
2,1993,1004,307.723,899.877487,10.334283,92.647501,289.686822,1477.709719,1,-1,0.000000,-0.002496
3,1994,1004,321.632,968.610089,4.387066,62.109581,252.017840,1094.266391,2,0,0.000000,-0.000000
4,1995,1004,338.012,668.644900,4.167967,48.314072,239.411707,1126.941572,4,0,0.000000,-0.000000
5,1996,1004,414.100,185.418981,3.006216,42.060579,203.376909,1038.972931,5,1,0.004320,-0.000000
6,1997,1004,468.400,31.137916,1.726458,31.004673,159.880012,930.978037,1,-1,0.000000,-0.003700
7,1998,1004,508.186,99.424227,2.610292,34.365018,220.755999,1442.550763,5,1,0.003039,-0.000000
8,1999,1004,511.267,98.975682,2.567503,29.989193,196.467615,1284.714194,2,0,0.000000,-0.000000
9,2000,1004,485.856,125.775950,2.067513,21.533410,156.374651,1065.691545,2,0,0.000000,-0.000000


In [215]:
#########################################################
# weighted protfolio setup for each year

portfolio_at.drop(columns = ['p20', 'p40', 'p60', 'p80'], axis = 1, inplace=True)
portfolio_at['group_tag'].replace({1: 'L', 5: 'H'}, inplace=True)
portfolio_at['weight']=portfolio['weight_l']+portfolio['weight_s']
print(portfolio_at.head())

   fyear  gvkey       at         p_at group_tag  trade  weight_l  weight_s  \
0   1991   1004  289.537  1378.922210         4      0       0.0 -0.000000   
1   1992   1004  265.036   811.602952         2      0       0.0 -0.000000   
2   1993   1004  307.723   899.877487         L     -1       0.0 -0.002496   
3   1994   1004  321.632   968.610089         2      0       0.0 -0.000000   
4   1995   1004  338.012   668.644900         4      0       0.0 -0.000000   

     weight  
0  0.000000  
1  0.000000  
2 -0.002633  
3  0.000000  
4  0.002735  


In [216]:
portfolio_at.to_csv('portfolio_at.csv', index=False)

## Scaling - PPENT

In [217]:
percentiles_df = ppent_df.groupby('fyear')['p_ppent'].quantile([0.2, 0.4, 0.6, 0.8]).unstack().reset_index()
percentiles_df.columns = ['fyear', 'p20', 'p40', 'p60', 'p80']

# Merge percentiles back into the original dataset
ppent_df = ppent_df.merge(percentiles_df, on='fyear', how='left')

In [218]:
print(ppent_df.tail())

       fyear   gvkey   ppent    p_ppent       p20        p40         p60  \
15547   2013  315318  139.67  15.128708  0.802541  13.416382   88.682435   
15548   2014  315318  175.00   8.746302  0.787324  11.886396   97.491899   
15549   2015  315318  491.60   8.347178  1.378901  15.852882  104.121718   
15550   2016  315318  460.50  13.373859  0.949676  15.249470  114.227958   
15551   2017  315318  452.30   9.463939  1.199172  16.556415  109.129279   

              p80  
15547  406.936683  
15548  449.196388  
15549  440.909617  
15550  400.194322  
15551  392.865539  


In [219]:
##################################
# Function to map e_at value to group tag
def map_to_group_tag(scale_value, percentiles):
    if scale_value <= percentiles['p20']:
        return 1
    elif scale_value <= percentiles['p40']:
        return 2
    elif scale_value <= percentiles['p60']:
        return 3
    elif scale_value <= percentiles['p80']:
        return 4
    else:
        return 5  # Adjust this based on your requirements

# Add a new column 'group_tag' to the DataFrame based on 'e_at' percentiles for each year
group_tags = []

for year in ppent_df['fyear'].unique():
    year_data = ppent_df[ppent_df['fyear'] == year]
    percentiles = year_data[['p20', 'p40', 'p60', 'p80']].iloc[0].to_dict()
    group_tags.extend(year_data['p_ppent'].apply(map_to_group_tag, args=(percentiles,)))

ppent_df['group_tag'] = group_tags

ppent_df.head()

,fyear,gvkey,ppent,p_ppent,p20,p40,p60,p80,group_tag
0,1991,1004,60.422,6607.676012,20.252730,146.492011,535.018251,2521.741690,5
1,1992,1004,56.052,3837.579391,12.231291,119.950721,473.466885,2445.336967,2
2,1993,1004,54.783,5054.725006,11.057642,102.558315,405.805636,1819.366195,1
3,1994,1004,56.596,5504.558626,6.684491,70.855117,296.764263,1661.600331,3
4,1995,1004,54.831,4121.938320,5.110534,64.599877,302.056080,1524.847134,4


In [220]:
###################################################################
# Count occurrences of each group for each year
group_counts = ppent_df.groupby(['fyear', 'group_tag']).size().unstack(fill_value=0)
print(group_counts)

group_tag    1    2    3    4    5
fyear                             
1991       106  127  116  112  109
1992       112  117  118  126  112
1993       119  118  124  113  107
1994       130  122   94  105  123
1995       109  113  116  111  118
1996       105  107  113  115  130
1997       118   95  112  121  108
1998       109  121  127  118  116
1999       120  104  114  133   96
2000       121  100  124  114   97
2001       107  115  103  121  111
2002       111   96  107  111  104
2003        96  121   93  105  115
2004       104   99  108   97  121
2005        97  110  116  101   98
2006       104   88   97  102  112
2007       100   97   89   98  109
2008        90  103   83  100  104
2009       105   99   81   82  104
2010        97   94   95   73  111
2011       111   82   92   84   96
2012        86   92  111   81   87
2013        92   89  106   86   77
2014        96   84   79   87   91
2015        80   77   77   83  100
2016        84   65   83   75   91
2017        72   85 

In [221]:

#########################################
# portfolio formation
# Add a new column 'trade' assigning 1 to Group 5 and -1 to Group 1 for each year
ppent_df['trade'] = ppent_df['group_tag'].apply(lambda x: 1 if x == 5 else (-1 if x == 1 else 0))
sum_at_l = ppent_df[ppent_df['trade'] == 1].groupby('fyear')['ppent'].sum()
sum_at_s = ppent_df[ppent_df['trade'] == -1].groupby('fyear')['ppent'].sum()
# calculate wights each year
for year in ppent_df['fyear'].unique():
    year_data = ppent_df[ppent_df['fyear'] == year]
    ppent_df.loc[ppent_df['fyear'] == year, 'weight_l'] = year_data['ppent']* (year_data['trade'] == 1) /sum_at_l.iloc[year-1991]
    ppent_df.loc[ppent_df['fyear'] == year, 'weight_s'] = -year_data['ppent']* (year_data['trade'] == -1) /sum_at_s.iloc[year-1991]
portfolio_ppent = ppent_df
portfolio_ppent.dropna(inplace=True)
portfolio_ppent.head(20)

,fyear,gvkey,ppent,p_ppent,p20,p40,p60,p80,group_tag,trade,weight_l,weight_s
0,1991,1004,60.422,6607.676012,20.252730,146.492011,535.018251,2521.741690,5,1,0.000674,-0.000000
1,1992,1004,56.052,3837.579391,12.231291,119.950721,473.466885,2445.336967,2,0,0.000000,-0.000000
2,1993,1004,54.783,5054.725006,11.057642,102.558315,405.805636,1819.366195,1,-1,0.000000,-0.000474
3,1994,1004,56.596,5504.558626,6.684491,70.855117,296.764263,1661.600331,3,0,0.000000,-0.000000
4,1995,1004,54.831,4121.938320,5.110534,64.599877,302.056080,1524.847134,4,0,0.000000,-0.000000
5,1996,1004,71.108,1079.794116,3.573692,40.537462,267.981592,1230.390341,5,1,0.000437,-0.000000
6,1997,1004,82.905,175.924251,2.316189,34.448131,219.317232,1069.841166,1,-1,0.000000,-0.000564
7,1998,1004,104.012,485.770873,2.279000,36.721073,228.140713,1095.009721,4,0,0.000000,-0.000000
8,1999,1004,110.003,460.014727,2.948187,31.706788,236.536561,1012.153897,2,0,0.000000,-0.000000
9,2000,1004,108.907,561.111774,2.018873,25.103478,223.876574,970.893925,2,0,0.000000,-0.000000


In [222]:
#########################################################
# weighted protfolio setup for each year

portfolio_ppent.drop(columns = ['p20', 'p40', 'p60', 'p80'], axis = 1, inplace=True)
portfolio_ppent['group_tag'].replace({1: 'L', 5: 'H'}, inplace=True)
portfolio_ppent['weight']=portfolio['weight_l']+portfolio['weight_s']
print(portfolio_ppent.head())
#portfolio.to_csv('portfolio.csv', index=False)

   fyear  gvkey   ppent      p_ppent group_tag  trade  weight_l  weight_s  \
0   1991   1004  60.422  6607.676012         H      1  0.000674 -0.000000   
1   1992   1004  56.052  3837.579391         2      0  0.000000 -0.000000   
2   1993   1004  54.783  5054.725006         L     -1  0.000000 -0.000474   
3   1994   1004  56.596  5504.558626         3      0  0.000000 -0.000000   
4   1995   1004  54.831  4121.938320         4      0  0.000000 -0.000000   

     weight  
0  0.000000  
1  0.000000  
2 -0.002633  
3  0.000000  
4  0.002735  


In [223]:
portfolio_ppent.to_csv('portfolio_ppent.csv', index=False)

# Scaling- SALE

In [224]:
percentiles_df = sale_df.groupby('fyear')['p_sale'].quantile([0.2, 0.4, 0.6, 0.8]).unstack().reset_index()
percentiles_df.columns = ['fyear', 'p20', 'p40', 'p60', 'p80']

# Merge percentiles back into the original dsaleaset
sale_df = sale_df.merge(percentiles_df, on='fyear', how='left')

print(sale_df.tail())

       fyear   gvkey      sale    p_sale       p20       p40        p60  \
15547   2013  315318   745.951  6.874137  0.168571  3.300643  21.842791   
15548   2014  315318   843.200  3.873037  0.179929  2.952832  29.274828   
15549   2015  315318  2542.300  2.890588  0.250845  4.246685  33.186237   
15550   2016  315318  3585.900  3.903840  0.183403  3.989881  34.272661   
15551   2017  315318  3775.900  2.561410  0.326986  4.818726  33.500864   

              p80  
15547  175.086133  
15548  165.706412  
15549  184.511679  
15550  185.194939  
15551  190.855606  


In [225]:
##################################
# Function to map e_sale value to group tag
def map_to_group_tag(scale_value, percentiles):
    if scale_value <= percentiles['p20']:
        return 1
    elif scale_value <= percentiles['p40']:
        return 2
    elif scale_value <= percentiles['p60']:
        return 3
    elif scale_value <= percentiles['p80']:
        return 4
    else:
        return 5  # Adjust this based on your requirements

# Add a new column 'group_tag' to the DsaleaFrame based on 'e_sale' percentiles for each year
group_tags = []

for year in sale_df['fyear'].unique():
    year_dsalea = sale_df[sale_df['fyear'] == year]
    percentiles = year_dsalea[['p20', 'p40', 'p60', 'p80']].iloc[0].to_dict()
    group_tags.extend(year_dsalea['p_sale'].apply(map_to_group_tag, args=(percentiles,)))

sale_df['group_tag'] = group_tags

sale_df.head()

,fyear,gvkey,sale,p_sale,p20,p40,p60,p80,group_tag
0,1991,1004,422.657,944.617030,5.739919,42.279801,159.098584,710.347801,5
1,1992,1004,382.780,561.952035,3.437469,34.851072,132.097380,627.487904,2
2,1993,1004,407.754,679.117801,3.443323,31.607985,108.447031,498.901311,1
3,1994,1004,451.395,690.162718,1.630245,21.679677,89.291220,405.745629,2
4,1995,1004,504.990,447.553417,1.420281,16.808399,84.559965,373.973420,4


In [226]:
###################################################################
# Count occurrences of each group for each year
group_counts = sale_df.groupby(['fyear', 'group_tag']).size().unstack(fill_value=0)
print(group_counts)

group_tag    1    2    3    4    5
fyear                             
1991       110  116  123  107  114
1992       115  117  115  119  119
1993       120  125  119  105  112
1994       134  112  104  106  118
1995       106  115  111  120  115
1996       101  119  106  121  123
1997       114  109  109  105  117
1998       109  117  138   95  132
1999       118  108  120  118  103
2000       118  102  116  116  104
2001       106  117  106  110  118
2002       112   96  114  103  104
2003        99  110  107   94  120
2004       109  100  100  111  109
2005        91  111  110  113   97
2006       106   87   92  108  110
2007        96   97   93  101  106
2008        85  100  100   90  105
2009       110   90   80   97   94
2010        95   99   90   87   99
2011       110   86   83   92   94
2012        85   92   91  116   73
2013        87  106   90   94   73
2014       100   87   76   85   89
2015        83   75   79   85   95
2016        84   60   88   85   81
2017        80   74 

In [227]:
#########################################
# portfolio formsaleion
# Add a new column 'trade' assigning 1 to Group 5 and -1 to Group 1 for each year
sale_df['trade'] = sale_df['group_tag'].apply(lambda x: 1 if x == 5 else (-1 if x == 1 else 0))
sum_sale_l = sale_df[sale_df['trade'] == 1].groupby('fyear')['sale'].sum()
sum_sale_s = sale_df[sale_df['trade'] == -1].groupby('fyear')['sale'].sum()
# calculsalee wights each year
for year in sale_df['fyear'].unique():
    year_dsalea = sale_df[sale_df['fyear'] == year]
    sale_df.loc[sale_df['fyear'] == year, 'weight_l'] = year_dsalea['sale']* (year_dsalea['trade'] == 1) /sum_sale_l.iloc[year-1991]
    sale_df.loc[sale_df['fyear'] == year, 'weight_s'] = -year_dsalea['sale']* (year_dsalea['trade'] == -1) /sum_sale_s.iloc[year-1991]
portfolio_sale = sale_df
portfolio_sale.dropna(inplace=True)
portfolio_sale.head(20)


,fyear,gvkey,sale,p_sale,p20,p40,p60,p80,group_tag,trade,weight_l,weight_s
0,1991,1004,422.657,944.617030,5.739919,42.279801,159.098584,710.347801,5,1,0.001616,-0.000000
1,1992,1004,382.780,561.952035,3.437469,34.851072,132.097380,627.487904,2,0,0.000000,-0.000000
2,1993,1004,407.754,679.117801,3.443323,31.607985,108.447031,498.901311,1,-1,0.000000,-0.001221
3,1994,1004,451.395,690.162718,1.630245,21.679677,89.291220,405.745629,2,0,0.000000,-0.000000
4,1995,1004,504.990,447.553417,1.420281,16.808399,84.559965,373.973420,4,0,0.000000,-0.000000
5,1996,1004,589.328,130.287378,1.074018,12.361733,72.135807,347.572157,5,1,0.002099,-0.000000
6,1997,1004,782.123,18.647962,0.680932,10.354110,58.850766,312.980118,1,-1,0.000000,-0.002206
7,1998,1004,918.036,55.037057,0.785604,11.255959,78.429785,437.646719,4,0,0.000000,-0.000000
8,1999,1004,1024.333,49.400927,0.757381,10.345937,71.436416,393.962494,2,0,0.000000,-0.000000
9,2000,1004,874.255,69.898371,0.538858,7.241400,56.319266,342.266730,2,0,0.000000,-0.000000


In [228]:
#########################################################
# weighted protfolio setup for each year

portfolio_sale.drop(columns = ['p20', 'p40', 'p60', 'p80'], axis = 1, inplace=True)
portfolio_sale['group_tag'].replace({1: 'L', 5: 'H'}, inplace=True)
portfolio_sale['weight']=portfolio['weight_l']+portfolio['weight_s']
print(portfolio_sale.head())

   fyear  gvkey     sale      p_sale group_tag  trade  weight_l  weight_s  \
0   1991   1004  422.657  944.617030         H      1  0.001616 -0.000000   
1   1992   1004  382.780  561.952035         2      0  0.000000 -0.000000   
2   1993   1004  407.754  679.117801         L     -1  0.000000 -0.001221   
3   1994   1004  451.395  690.162718         2      0  0.000000 -0.000000   
4   1995   1004  504.990  447.553417         4      0  0.000000 -0.000000   

     weight  
0  0.000000  
1  0.000000  
2 -0.002633  
3  0.000000  
4  0.002735  


In [229]:
portfolio_sale.to_csv('portfolio_sale.csv', index=False)

# Scaling- EQ

In [230]:
percentiles_df = eq_df.groupby('fyear')['p_eq'].quantile([0.2, 0.4, 0.6, 0.8]).unstack().reset_index()
percentiles_df.columns = ['fyear', 'p20', 'p40', 'p60', 'p80']

# Merge percentiles back into the original deqaset
eq_df = eq_df.merge(percentiles_df, on='fyear', how='left')

print(eq_df.tail())


       fyear   gvkey        eq       p_eq       p20       p40        p60  \
15547   2013  315318  1032.595  10.134232  0.429809  6.009031  52.608760   
15548   2014  315318  2459.600   4.483949  0.538792  6.664054  70.032665   
15549   2015  315318  2103.900   4.687864  0.612573  7.744914  63.179807   
15550   2016  315318  2736.100   5.837994  0.458050  8.166548  79.823077   
15551   2017  315318  2743.100   3.938098  0.534022  8.982532  72.904126   

              p80  
15547  327.803496  
15548  355.957327  
15549  363.886925  
15550  356.111287  
15551  337.333725  


In [231]:
##################################
# Function to map e_eq value to group tag
def map_to_group_tag(scale_value, percentiles):
    if scale_value <= percentiles['p20']:
        return 1
    elif scale_value <= percentiles['p40']:
        return 2
    elif scale_value <= percentiles['p60']:
        return 3
    elif scale_value <= percentiles['p80']:
        return 4
    else:
        return 5  # Adjust this based on your requirements

# Add a new column 'group_tag' to the DeqaFrame based on 'e_eq' percentiles for each year
group_tags = []

for year in eq_df['fyear'].unique():
    year_deqa = eq_df[eq_df['fyear'] == year]
    percentiles = year_deqa[['p20', 'p40', 'p60', 'p80']].iloc[0].to_dict()
    group_tags.extend(year_deqa['p_eq'].apply(map_to_group_tag, args=(percentiles,)))

eq_df['group_tag'] = group_tags

eq_df.head()

,fyear,gvkey,eq,p_eq,p20,p40,p60,p80,group_tag
0,1991,1004,196.737,2029.353909,13.958837,118.783383,436.102866,1962.596529,5
1,1992,1004,189.216,1136.817182,7.295346,100.699969,390.906533,1956.977696,2
2,1993,1004,189.488,1461.374863,8.591631,88.366806,332.693503,1622.049640,1
3,1994,1004,197.119,1580.446329,3.816066,55.920198,250.560553,1256.723825,3
4,1995,1004,204.635,1104.454272,4.282501,48.996521,251.110936,1259.087487,4


In [232]:
###################################################################
# Count occurrences of each group for each year
group_counts = eq_df.groupby(['fyear', 'group_tag']).size().unstack(fill_value=0)
print(group_counts)

group_tag    1    2    3    4    5
fyear                             
1991       112  123  112  103  120
1992       114  122  105  125  119
1993       123  123  115  109  111
1994       130  119  102   99  124
1995       106  114  112  123  112
1996        99  109  116  129  117
1997       112  110  114  106  112
1998       101  130  125  108  127
1999       122  105  114  125  101
2000       122  107  113  113  101
2001       102  111  118  109  117
2002       108  103  108  108  102
2003       100  108  110   95  117
2004       107  104  103  110  105
2005        99  106  107  109  101
2006       106   85   99  101  112
2007       100   91  106   89  107
2008        87   95   97  103   98
2009       108   91   79   89  104
2010        91   96  101   80  102
2011       117   78   87   96   87
2012        88   98   94  106   71
2013        87   91  110   80   82
2014        99   87   72   88   91
2015        77   77   77   87   99
2016        80   66   88   76   88
2017        79   69 

In [233]:
#########################################
# portfolio formeqion
# Add a new column 'trade' assigning 1 to Group 5 and -1 to Group 1 for each year
eq_df['trade'] = eq_df['group_tag'].apply(lambda x: 1 if x == 5 else (-1 if x == 1 else 0))
sum_eq_l = eq_df[eq_df['trade'] == 1].groupby('fyear')['eq'].sum()
sum_eq_s = eq_df[eq_df['trade'] == -1].groupby('fyear')['eq'].sum()
# calculeqe wights each year
for year in eq_df['fyear'].unique():
    year_deqa = eq_df[eq_df['fyear'] == year]
    eq_df.loc[eq_df['fyear'] == year, 'weight_l'] = year_deqa['eq']* (year_deqa['trade'] == 1) /sum_eq_l.iloc[year-1991]
    eq_df.loc[eq_df['fyear'] == year, 'weight_s'] = -year_deqa['eq']* (year_deqa['trade'] == -1) /sum_eq_s.iloc[year-1991]
portfolio_eq = eq_df
portfolio_eq.dropna(inplace=True)
portfolio_eq.head(20)

,fyear,gvkey,eq,p_eq,p20,p40,p60,p80,group_tag,trade,weight_l,weight_s
0,1991,1004,196.737,2029.353909,13.958837,118.783383,436.102866,1962.596529,5,1,0.002107,-0.000000
1,1992,1004,189.216,1136.817182,7.295346,100.699969,390.906533,1956.977696,2,0,0.000000,-0.000000
2,1993,1004,189.488,1461.374863,8.591631,88.366806,332.693503,1622.049640,1,-1,0.000000,-0.001544
3,1994,1004,197.119,1580.446329,3.816066,55.920198,250.560553,1256.723825,3,0,0.000000,-0.000000
4,1995,1004,204.635,1104.454272,4.282501,48.996521,251.110936,1259.087487,4,0,0.000000,-0.000000
5,1996,1004,269.259,285.160385,2.718520,34.347910,212.108439,1101.115835,5,1,0.002541,-0.000000
6,1997,1004,300.850,48.479309,1.391590,28.896646,168.063501,889.093549,1,-1,0.000000,-0.001741
7,1998,1004,326.035,154.971092,2.001806,30.633180,189.783524,1148.709882,5,1,0.001053,-0.000000
8,1999,1004,339.515,149.044961,2.438967,27.988514,195.973669,1031.196021,2,0,0.000000,-0.000000
9,2000,1004,340.212,179.620354,0.794966,17.464281,149.506423,822.550664,2,0,0.000000,-0.000000


In [234]:
#########################################################
# weighted protfolio setup for each year

portfolio_eq.drop(columns = ['p20', 'p40', 'p60', 'p80'], axis = 1, inplace=True)
portfolio_eq['group_tag'].replace({1: 'L', 5: 'H'}, inplace=True)
portfolio_eq['weight']=portfolio['weight_l']+portfolio['weight_s']
print(portfolio_eq.head())


   fyear  gvkey       eq         p_eq group_tag  trade  weight_l  weight_s  \
0   1991   1004  196.737  2029.353909         H      1  0.002107 -0.000000   
1   1992   1004  189.216  1136.817182         2      0  0.000000 -0.000000   
2   1993   1004  189.488  1461.374863         L     -1  0.000000 -0.001544   
3   1994   1004  197.119  1580.446329         3      0  0.000000 -0.000000   
4   1995   1004  204.635  1104.454272         4      0  0.000000 -0.000000   

     weight  
0  0.000000  
1  0.000000  
2 -0.002633  
3  0.000000  
4  0.002735  


In [235]:
portfolio_eq.to_csv('portfolio_eq.csv', index=False)